__AUTHORS__:
  - Théo Perinet (22172 - theo.perinet)
  - Mathieu Rivier (23553 - mathieu.rivier)
  - Marc Monteil (23742 - marc.monteil)

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#BigData---Final-Project" data-toc-modified-id="BigData---Final-Project-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>BigData - Final Project</a></span><ul class="toc-item"><li><span><a href="#Loading-The-Data" data-toc-modified-id="Loading-The-Data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Loading The Data</a></span></li><li><span><a href="#Exploring-The-Data" data-toc-modified-id="Exploring-The-Data-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Exploring The Data</a></span></li><li><span><a href="#Analysis" data-toc-modified-id="Analysis-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Analysis</a></span></li></ul></li></ul></div>

# BigData - Final Project

###### To Use when you are on google collab
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz  
!tar xf spark-3.2.1-bin-hadoop2.7.tgz
!pip install -q findspark

###### TO USE WHEN YOU ARE ON GOOGLE COLLAB
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"
import findspark
findspark.init()

from google.colab import drive
drive.mount('/content/drive')

## Loading The Data

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark_application_name = "WannaFlop_Project"

In [3]:
spark = (SparkSession.builder.appName(spark_application_name).getOrCreate())

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/22 17:07:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/22 17:07:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Exploring The Data

In [4]:
from pyspark.sql.functions import col,isnan,when,count
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.types import DoubleType, IntegerType, StringType, DateType, StructType,StructField
from pyspark.sql.functions import desc
import pyspark.sql.functions as func

In [5]:
class read_info(object):
    def __init__(self, file_path, header=False, delimiter=';', schema=None):
        self.file_path = file_path
        self.header = header
        self.delimiter = delimiter
        self.schema = schema

        self.df = self._load_df()

        #self.df_abstract = self._get_df_abstract()

    def __repr__(self):
        return f"{self._nb_rows()} \n{self.df.printSchema()} \n{self.get_df_abstract()}\n {self.show_missing()}\n{self._get_stats()}"

    def show_missing(self):
        print("Missing Data per column:")
        self._count_missing().show()

    def _get_num_cols(self):
        num_cols = [
            f.name for f in self.df.schema.fields
            if isinstance(f.dataType, DoubleType) or
            isinstance(f.dataType, IntegerType)
        ]
        
        return num_cols
    def _get_rounded_df(self):
        rounded_df = self.df
        dbl_cols = self._get_num_cols()
        for col in dbl_cols:
            rounded_df = rounded_df.withColumn(col, func.round('high'))

        return rounded_df

    def get_df_abstract(self):
        rounded_df = self._get_rounded_df()

        # First 40 rows
        print("First 40 rows:")
        rounded_df.show(40)

        # Last 40 rows
        print("Last 40 rows:")
        rounded_df = rounded_df.withColumn(
            "index", monotonically_increasing_id()
        )
        rounded_df.orderBy(desc("index")).drop("index").show(40)

    def _get_periodicity(self):
        self.df['data'][0]

    def _nb_rows(self):
        # Number of total rows
        print("Number of rows: " + str(self.df.count()) + "\n")

    def _handle_csv(self):
        '''
        @description: Read the csv file and return a Spark DataFrame

        @arg csv_file_path: Path to the csv file
        @arg header: boolean whether to load a header or not
        @arg delimiter: which delimiter to use by default
        '''
        return spark.read.option("inferSchema", "true").option("nullValue", "null").csv(
            self.file_path,
            sep=self.delimiter,
            schema=self.schema,
            header=self.header,
        )
    
    def _handle_json(self):
        return spark.read.json(self.file_path)

    def _load_df(self):
        ####### ADD TRY CATCH #####
        extension = self.file_path.split(".")[-1]

        df = None
        if extension == 'json':
            df = self._handle_json()
        elif extension == 'csv':
            df = self._handle_csv()

        return df

    def _count_missing(self):
        cols = self.df.columns
        cols.remove('Date')
        return self.df.select(
            [
                count(when(isnan(c) | col(c).isNull(), c)).alias(c)
                for c in cols
            ]
        )
        #.show()
        
    def _get_stats(self):
        self.df.summary().show()

In [6]:
print(h)
h.remove('Date')
print(h)

NameError: name 'h' is not defined

In [ ]:
amzn_schema = StructType([
    StructField('Date', DateType(), True),
    StructField('High', DoubleType(), True),
    StructField('Low', DoubleType(), True),
    StructField('Open', DoubleType(), True),
    StructField('Close', DoubleType(), True),
    StructField('Volume', IntegerType(), True),
    StructField('Adj Close', DoubleType(), True),
    StructField('company_name', StringType(), True)
])

In [ ]:
AMZN = read_info('stocks_data/AMAZON.csv', header=True, delimiter=',', schema=amzn_schema)

In [ ]:
print(AMZN)

In [ ]:
##### A FAIRE !!!! UN SCHEMA !!!!!

In [ ]:
AMZN.get_df_abstract()

In [ ]:
AMZN.show_missing()

In [ ]:
AMZN._get_stats()

In [ ]:
AMZN.df.withColumn("test", 
              func.datediff(AMZN.df["date"][0], AMZN.df["date"][1])).show()

In [ ]:
AMZN.df["Date"].getItem(2)

In [ ]:
AMZN.df.first()['Date']

In [ ]:
AMZN.df.__get_item(0)

In [ ]:
AMZN.df.second()['Date']

In [ ]:
func.getrows(AMZN.df, rownums=[0, 2]).collect()

In [ ]:
AMZN.df[0].__getitem__("Date").first()

In [ ]:
AMZN.df[0]

In [ ]:
AMZN.df.select('Date').show()

In [ ]:
AMZN.df


In [ ]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.window import Window



my_window = Window.partitionBy().orderBy("Date")

df = AMZN.df.withColumn("prev_value", F.lag(AMZN.df.Date).over(my_window))
df = df.withColumn("diff", F.when(F.isnull(F.datediff(df.Date, df.prev_value)), 0)
                              .otherwise(F.datediff(df.Date, df.prev_value)))

In [ ]:
df.select("diff").show()

In [ ]:
from pyspark.sql.functions import mean

In [ ]:
df.select(mean('diff')).first()[0]

In [ ]:
AMZN.df.stat.corr('High', 'Low')

TODO: Create function to compute per month week year

In [ ]:
AMZN.df.select(mean ("Close")).first()[0]

In [ ]:
def get_col_mean(df, col):
    return df.select(mean (col)).first()[0]

In [ ]:
get_col_mean(AMZN.df, "Close")

In [ ]:
AMZN.df.groupBy(func.weekofyear("day").alias("date_by_week")).agg(sum("Date"))

In [ ]:
AMZN.df.groupBy(func.weekofyear("day").alias("date_by_week")).agg(sum("Close")).orderBy("date_by_week").show()

In [ ]:
AMZN.df.withColumn("Date",func.date_sub(func.next_day(col("Date"),"sunday"),7)).groupBy("Date").agg(sum("Close").cast("int").alias("Close_total")).orderBy("week_strt_day").show()


In [ ]:
AMZN.df.groupBy("Date").select("Close").show()

In [ ]:
AMZN.df.groupBy(func.month("Date").alias("hour")).agg(mean("Close").alias("close_mean")).show()

In [ ]:
AMZN.df.groupBy(func.year("Date").alias("hour")).agg(mean("Close").alias("close_mean")).show()

In [ ]:
get_avg(AMZN.df, "Close", func.year)

In [ ]:
get_avg(AMZN.df, "Open", func.year)

In [ ]:
class Exploration(object):
    def __init__(self, df):
        self.df = df

    def get_oc_avg(self, fun):
        close = self._compute_avg(self.df, "Close", fun)
        opening = self._compute_avg(self.df, "Open", fun)

        return close.join(
            opening, opening.Open_new_time == close.Close_new_time, "inner"
        ).orderBy("Close_new_time").select(
            close.Close_new_time, close.Close_mean, opening.Open_mean
        )

    def _compute_avg(self, df, col, fun):
        return df.groupBy(fun("Date").alias(col + "_new_time")).agg(
            mean(col).alias(col + "_mean")
        )

In [72]:
exAMZN = Exploration(AMZN.df)

NameError: name 'Exploration' is not defined

In [ ]:
exAMZN.get_oc_avg(func.month).show()

In [7]:
exAMZN.get_oc_avg(func.year).show()

NameError: name 'exAMZN' is not defined

In [8]:
def get_price_change(period=None):
    df = AMZN.df
    if period:
        df= exAMZN.get_oc_avg(period)
   
    return  df.withColumn('diff', ( df['Close_mean'] - df['Open_mean'] ))

In [9]:
get_price_change(func.month).show()

NameError: name 'AMZN' is not defined

In [10]:
get_price_change(func.year).show()

NameError: name 'AMZN' is not defined

In [70]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [73]:
from stock import Stock

In [74]:
from pyspark.sql.types import DoubleType, IntegerType, StringType, DateType, StructType,StructField

In [75]:
amzn_schema = StructType([
    StructField('Date', DateType(), True),
    StructField('High', DoubleType(), True),
    StructField('Low', DoubleType(), True),
    StructField('Open', DoubleType(), True),
    StructField('Close', DoubleType(), True),
    StructField('Volume', DoubleType(), True),
    StructField('Adj Close', DoubleType(), True),
    StructField('company_name', StringType(), True)
])

In [76]:
AMZN = Stock('stocks_data/AMAZON.csv', header=True, delimiter=',', schema=amzn_schema)

In [77]:
AMZN.explore

root
 |-- Date: date (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Open: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- company_name: string (nullable = true)

First 40 rows:
+----------+-----+-----+-----+-----+---------+---------+------------+
|      Date| High|  Low| Open|Close|   Volume|Adj Close|company_name|
+----------+-----+-----+-----+-----+---------+---------+------------+
|2017-01-03|759.0|748.0|758.0|754.0|3521100.0|    754.0|      AMAZON|
|2017-01-04|760.0|754.0|758.0|757.0|2510500.0|    757.0|      AMAZON|
|2017-01-05|782.0|760.0|762.0|780.0|5830100.0|    780.0|      AMAZON|
|2017-01-06|799.0|778.0|782.0|796.0|5986200.0|    796.0|      AMAZON|
|2017-01-09|802.0|792.0|798.0|797.0|3446100.0|    797.0|      AMAZON|
|2017-01-10|798.0|790.0|797.0|796.0|2558400.0|    796.0|      AMAZON|
|2017-01-11|800.0|790.0|794.0|799.0|2992800.0|  

AttributeError: 'Explore' object has no attribute '_nb_rows'

In [78]:
AMZN.explore._nb_rows()

AttributeError: 'Explore' object has no attribute '_nb_rows'

In [79]:
AMZN.explore._print_schema()

root
 |-- Date: date (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Open: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- company_name: string (nullable = true)



In [80]:
AMZN.explore.get_df_abstract()

First 40 rows:
+----------+-----+-----+-----+-----+---------+---------+------------+
|      Date| High|  Low| Open|Close|   Volume|Adj Close|company_name|
+----------+-----+-----+-----+-----+---------+---------+------------+
|2017-01-03|759.0|748.0|758.0|754.0|3521100.0|    754.0|      AMAZON|
|2017-01-04|760.0|754.0|758.0|757.0|2510500.0|    757.0|      AMAZON|
|2017-01-05|782.0|760.0|762.0|780.0|5830100.0|    780.0|      AMAZON|
|2017-01-06|799.0|778.0|782.0|796.0|5986200.0|    796.0|      AMAZON|
|2017-01-09|802.0|792.0|798.0|797.0|3446100.0|    797.0|      AMAZON|
|2017-01-10|798.0|790.0|797.0|796.0|2558400.0|    796.0|      AMAZON|
|2017-01-11|800.0|790.0|794.0|799.0|2992800.0|    799.0|      AMAZON|
|2017-01-12|814.0|800.0|800.0|814.0|4873900.0|    814.0|      AMAZON|
|2017-01-13|822.0|811.0|814.0|817.0|3791900.0|    817.0|      AMAZON|
|2017-01-17|816.0|803.0|816.0|810.0|3670500.0|    810.0|      AMAZON|
|2017-01-18|812.0|804.0|810.0|807.0|2354200.0|    807.0|      AMAZON|
|2017

In [81]:
AMZN.explore.get_missing()

Missing Data per column:
+----+---+----+-----+------+---------+------------+
|High|Low|Open|Close|Volume|Adj Close|company_name|
+----+---+----+-----+------+---------+------------+
|   0|  0|   0|    0|     0|        0|           0|
+----+---+----+-----+------+---------+------------+



In [ ]:
AMZN.explore.get_stats()

## Analysis

In [82]:
AMZN.analysis.get_oc_avg("day").show()

+----------+-----------------+-----------------+
|      Date|        Open_mean|       Close_mean|
+----------+-----------------+-----------------+
|2017-01-03|757.9199829101562|753.6699829101562|
|2017-01-04|758.3900146484375|757.1799926757812|
|2017-01-05|761.5499877929688|780.4500122070312|
|2017-01-06|782.3599853515625| 795.989990234375|
|2017-01-09|            798.0|796.9199829101562|
|2017-01-10|796.5999755859375|795.9000244140625|
|2017-01-11|793.6599731445312|  799.02001953125|
|2017-01-12|800.3099975585938|813.6400146484375|
|2017-01-13|814.3200073242188|817.1400146484375|
|2017-01-17|815.7000122070312| 809.719970703125|
|2017-01-18|            809.5|  807.47998046875|
|2017-01-19|            810.0|809.0399780273438|
|2017-01-20| 815.280029296875|808.3300170898438|
|2017-01-23|806.7999877929688|817.8800048828125|
|2017-01-24|            822.0|822.4400024414062|
|2017-01-25|825.7899780273438|  836.52001953125|
|2017-01-26| 835.530029296875|839.1500244140625|
|2017-01-27|        

In [ ]:
AMZN.analysis.get_oc_avg("year").show()

In [83]:
AMZN.analysis.get_price_change("month").show()

+-----------------+------------------+
|Date_period_month|      price_change|
+-----------------+------------------+
|          2017-01| 65.55999755859375|
|          2017-02|  15.8299560546875|
|          2017-03|   33.489990234375|
|          2017-04|   36.989990234375|
|          2017-05| 66.82000732421875|
|          2017-06|-30.59002685546875|
|          2017-07| 14.99005126953125|
|          2017-08|  -15.510009765625|
|          2017-09|-22.85003662109375|
|          2017-10|  141.280029296875|
|          2017-11|  71.3499755859375|
|          2017-12|      -2.580078125|
|          2018-01| 278.8900146484375|
|          2018-02|   67.449951171875|
|          2018-03|  -66.260009765625|
|          2018-04|  148.510009765625|
|          2018-05|  66.4000244140625|
|          2018-06|    62.77001953125|
|          2018-07|   94.739990234375|
|          2018-08|    228.7099609375|
+-----------------+------------------+
only showing top 20 rows



In [ ]:
AMZN.analysis.get_price_change("year").show()

In [ ]:
AMZN.analysis._compute_avg(AMZN.df, "Close", "month").show()

In [ ]:
AMZN.analysis.get_daily_return(period="month").show()

In [ ]:
AMZN.analysis.get_daily_return(period="day").show()

In [84]:
AMZN.analysis.get_daily_return_rate(period="month", start_price=15.4, nb_shares=100).show()

+-------+-----------------------+
|   Date|daily_return_rate_month|
+-------+-----------------------+
|2017-01|     1644.1583014153669|
|2017-02|      1403.965040664501|
|2017-03|      778.0914048163146|
|2017-04|     -977.1027633447616|
|2017-05|      938.6064873957786|
|2017-06|    -2985.8333583142844|
|2017-07|       538.633272245377|
|2017-08|     -1176.170896231819|
|2017-09|    -1922.4033108005126|
|2017-10|      766.2272087375865|
|2017-11|     1561.8432209530274|
|2017-12|    -1324.0308885451077|
|2018-01|      4957.009659332405|
|2018-02|    -3084.4239282510875|
|2018-03|    -1164.1786064567941|
|2018-04|    -4948.9811016735675|
|2018-05|      2876.034931421588|
|2018-06|    -171.92890117694805|
|2018-07|     -1075.762558219423|
|2018-08|     3826.9318678095706|
+-------+-----------------------+
only showing top 20 rows



In [85]:
AMZN.analysis.get_daily_return_rate(period="year", start_price=15.4, nb_shares=100).show()

+----+----------------------+
|Date|daily_return_rate_year|
+----+----------------------+
|2017|    -70.52423296867548|
|2018|   -1523.7229223325576|
|2019|     287.6729971107621|
|2020|     93.60413614853928|
+----+----------------------+



In [ ]:
import glob

glob.glob("stocks_data/*.csv")

In [ ]:
from stocks import Stocks

In [86]:
stocks = Stocks(schema=amzn_schema)

NameError: name 'Stocks' is not defined

In [87]:
stocks.get_max_daily_return()

NameError: name 'stocks' is not defined

In [88]:
AMZN.df.select('Date').show()

+----------+
|      Date|
+----------+
|2017-01-03|
|2017-01-04|
|2017-01-05|
|2017-01-06|
|2017-01-09|
|2017-01-10|
|2017-01-11|
|2017-01-12|
|2017-01-13|
|2017-01-17|
|2017-01-18|
|2017-01-19|
|2017-01-20|
|2017-01-23|
|2017-01-24|
|2017-01-25|
|2017-01-26|
|2017-01-27|
|2017-01-30|
|2017-01-31|
+----------+
only showing top 20 rows



In [89]:
from pyspark.sql.functions import lit,row_number,col
from pyspark.sql.window import Window

w = Window().partitionBy(lit('Date')).orderBy(lit('Date'))

df1 = AMZN.df.withColumn("row_num", row_number().over(w))

df1.filter(col("row_num").between(1,2)).show()

+----------+-----------------+-----------------+-----------------+-----------------+---------+-----------------+------------+-------+
|      Date|             High|              Low|             Open|            Close|   Volume|        Adj Close|company_name|row_num|
+----------+-----------------+-----------------+-----------------+-----------------+---------+-----------------+------------+-------+
|2017-01-03| 758.760009765625|747.7000122070312|757.9199829101562|753.6699829101562|3521100.0|753.6699829101562|      AMAZON|      1|
|2017-01-04|759.6799926757812|754.2000122070312|758.3900146484375|757.1799926757812|2510500.0|757.1799926757812|      AMAZON|      2|
+----------+-----------------+-----------------+-----------------+-----------------+---------+-----------------+------------+-------+



In [90]:
AMZN.df.show()

+----------+-----------------+-----------------+-----------------+-----------------+---------+-----------------+------------+
|      Date|             High|              Low|             Open|            Close|   Volume|        Adj Close|company_name|
+----------+-----------------+-----------------+-----------------+-----------------+---------+-----------------+------------+
|2017-01-03| 758.760009765625|747.7000122070312|757.9199829101562|753.6699829101562|3521100.0|753.6699829101562|      AMAZON|
|2017-01-04|759.6799926757812|754.2000122070312|758.3900146484375|757.1799926757812|2510500.0|757.1799926757812|      AMAZON|
|2017-01-05|782.4000244140625| 760.260009765625|761.5499877929688|780.4500122070312|5830100.0|780.4500122070312|      AMAZON|
|2017-01-06|799.4400024414062|  778.47998046875|782.3599853515625| 795.989990234375|5986200.0| 795.989990234375|      AMAZON|
|2017-01-09|  801.77001953125|  791.77001953125|            798.0|796.9199829101562|3446100.0|796.9199829101562|      

In [91]:
from pyspark.sql.window import Window


SECS_IN_DAY = 86400
period = period * SECS_IN_DAY

df = AMZN.df.withColumn('Date', AMZN.df.Date.cast('timestamp'))
window = Window().partitionBy(lit('Date')).orderBy(F.col("Date").cast('long')).rangeBetween(-period, 0)

df = df.withColumn('moving_average', func.avg("Close").over(window)).select('Date', 'moving_average')

NameError: name 'period' is not defined

In [92]:
df.show()

+-------------------+-----------------+
|               Date|   moving_average|
+-------------------+-----------------+
|2017-01-03 00:00:00|753.6699829101562|
|2017-01-04 00:00:00|755.4249877929688|
|2017-01-05 00:00:00|763.7666625976562|
|2017-01-06 00:00:00|771.8224945068359|
|2017-01-09 00:00:00|   776.8419921875|
|2017-01-10 00:00:00|780.0183308919271|
|2017-01-11 00:00:00| 787.576670328776|
|2017-01-12 00:00:00|796.9866739908854|
|2017-01-13 00:00:00|803.1016743977865|
|2017-01-17 00:00:00|807.0840087890625|
|2017-01-18 00:00:00|            809.4|
|2017-01-19 00:00:00|811.4039916992188|
|2017-01-20 00:00:00|   810.3419921875|
|2017-01-23 00:00:00| 810.489990234375|
|2017-01-24 00:00:00|812.4816589355469|
|2017-01-25 00:00:00|816.9483337402344|
|2017-01-26 00:00:00|822.2266743977865|
|2017-01-27 00:00:00|826.6816813151041|
|2017-01-30 00:00:00| 830.356679280599|
|2017-01-31 00:00:00|831.2900085449219|
+-------------------+-----------------+
only showing top 20 rows



In [93]:
AMZN.df.show()

+----------+-----------------+-----------------+-----------------+-----------------+---------+-----------------+------------+
|      Date|             High|              Low|             Open|            Close|   Volume|        Adj Close|company_name|
+----------+-----------------+-----------------+-----------------+-----------------+---------+-----------------+------------+
|2017-01-03| 758.760009765625|747.7000122070312|757.9199829101562|753.6699829101562|3521100.0|753.6699829101562|      AMAZON|
|2017-01-04|759.6799926757812|754.2000122070312|758.3900146484375|757.1799926757812|2510500.0|757.1799926757812|      AMAZON|
|2017-01-05|782.4000244140625| 760.260009765625|761.5499877929688|780.4500122070312|5830100.0|780.4500122070312|      AMAZON|
|2017-01-06|799.4400024414062|  778.47998046875|782.3599853515625| 795.989990234375|5986200.0| 795.989990234375|      AMAZON|
|2017-01-09|  801.77001953125|  791.77001953125|            798.0|796.9199829101562|3446100.0|796.9199829101562|      

In [106]:
AMZN.analysis.moving_average()

+----------+--------------------+
|      Date|moving_average_Close|
+----------+--------------------+
|2017-01-03|   753.6699829101562|
|2017-01-04|   755.4249877929688|
|2017-01-05|   763.7666625976562|
|2017-01-06|   771.8224945068359|
|2017-01-09|   782.6349945068359|
|2017-01-10|   792.3150024414062|
|2017-01-11|   796.9575042724609|
|2017-01-12|   801.3700103759766|
|2017-01-13|   804.5240112304688|
|2017-01-17|               813.5|
|2017-01-18|   811.4466552734375|
|2017-01-19|   808.7466430664062|
|2017-01-20|   808.6424865722656|
|2017-01-23|   810.6824951171875|
|2017-01-24|   814.4225006103516|
|2017-01-25|   821.2925109863281|
|2017-01-26|   828.9975128173828|
|2017-01-27|   830.3520141601563|
|2017-01-30|   835.4550170898438|
|2017-01-31|   832.1950073242188|
+----------+--------------------+
only showing top 20 rows

